- 대안 생성자
- 메모리 절약 slots / gc.collect()
<!-- - repr() / str()의 차이 -->

# Chapter9. 파이썬스러운 객체
> 절대로 결코! 앞에 언더바 두 개를 사용하지 말라. 이것은 짜증스러울 정도로 개인적인 이름이다. - 이안 비킹(Ian Bicking)

파이썬 데이터 모델 덕분에 사용자가 정의한 자료형도 내장 자료형과 마찬가지로 자연스럽게 동작할 수 있다. 상속하지 않고도 *덕 타이핑 매커니즘을 통해 모든 것이 가능하다.
사용자 정의 클래스를 만들어 보자.

사용자 정의 클래스를 만들기 위해서 다음과 같은방법을 설명한다.
- `repr()`, `bytes()` 등 객체를 다르 방식으로 표현하는 내장 함수의 지원
- 클래스 메서드로 대안 생성자 구현
- `format()` 내장 함수와 `str.format()` 메서드에서 사용하는 포맷 언어 확장
- 읽기 전용 접근만 허용하는 속성 제공
- 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
- `__slots__`를 이용해서 메모리 절약하기

<details> 
<summary> *덕 타이핑 (Duck Typing) </summary>
- 오리라고 부르면 오리가 됨)
```python
# what is duck typing? (https://en.wikipedia.org/wiki/Duck_typing)
# - 사람이 오리인척 하면 오리라고 봐도 된다? 
#   즉, 미리 타입을 정해놓지 않고, 실행 시점에 타입을 결정하는 것을 의미한다.
class Duck:
    def quack(self): 
        print("꽥꽥!")
    def feathers(self):
        print("오리에게 흰색, 회색 깃털이 있습니다.")

class Person:
    def quack(self): 
        print("이 사람이 오리를 흉내내네요.")
    def feathers(self):
        print("사람은 바닥에서 깃털을 주워서 보여 줍니다.")

def in_the_forest(duck):
    duck.quack()
    duck.feathers()


in_the_forest(Duck())
in_the_forest(Person())

--------------------------------------------------------------------------
꽥꽥!
오리에게 흰색, 회색 깃털이 있습니다.
이 사람이 오리를 흉내내네요.
사람은 바닥에서 깃털을 주워서 보여 줍니다.
```
</details>

## 유클리드 벡터형
- `@classmethod`와 `@staticmethod`를 언제 사용할 것인가?
- 파이썬에서 비공개 및 보호된 송성, 사용법, 관계, 한계

## 9.1 객체 표현

- `repr()` 내장 함수는 객체를 **개발자**가 보기 좋은 형태의 문자열로 변환한다.
- `str()` 내장 함수는 객체를 **사용자**가 보기 좋은 형태의 문자열로 변환한다.

In [2]:
from array import array
import math

class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):       # __iter__ 를 통해서 iterable 하게 만들어 준다. -> 언패킹 가능
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return f"repr-> {class_name}({self.x!r}, {self.y!r})"
        # return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return "str-> " + str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)       # hypot = sqrt(x**2 + y**2)) 빗변의 길이를 구함 (즉 벡터간의 거리를 의미함)

    def __bool__(self):
        return bool(abs(self))

In [3]:
v1 = Vector(3, 4)

print(v1.x, v1.y)

3.0 4.0


In [4]:
print(*v1)      # *v1 을 통해서 언패킹 가능

3.0 4.0


In [5]:
x, y = v1
x, y

(3.0, 4.0)

In [6]:
print(v1)       # print는 str을 호출한다.

str-> (3.0, 4.0)


In [7]:
v1

repr-> Vector(3.0, 4.0)

In [8]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [9]:
abs(v1)

5.0

In [10]:
bool(v1), bool(Vector(0,0))

(True, False)

In [11]:
for x in v1:
    print(x)

3.0
4.0


In [12]:
print(v1 == [3,4])      # 내부에서 tuple로 처리하기 때문에 True 반환
print(v1 == (3,4))

True
True


## 9.3 대안 생성자
Vector를 bytes로 변환하는 메서드가 있으니, byte를 Vecotr로 변환할 수 있으면 좋겠다.  
array.array의 표준 라이브러리에서 frombytes()라는 메서드를 사용해 byte를 Vector로 만드는 메서드를 클래스 메서드로 추가해보자.
```python
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(*memv)
```

In [13]:
from array import array
import math

class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):       # __iter__ 를 통해서 iterable 하게 만들어 준다. -> 언패킹 가능
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return f"repr-> {class_name}({self.x!r}, {self.y!r})"
        # return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return "str-> " + str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)       # hypot = sqrt(x**2 + y**2)) 빗변의 길이를 구함 (즉 벡터간의 거리를 의미함)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [14]:
v1_clone = Vector.frombytes(bytes(v1))
v1_clone

repr-> Vector(3.0, 4.0)

In [15]:
v1 == v1_clone

True

## 9.4 @classmethod와 @staticmethod
- `@classmethod`는 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의한다. 클래스를 첫 번째 인수로 받으며, 위와 같이 관습적으로 `cls`라는 이름을 사용한다.
- `@staticmethod`는 함수처럼 작동한다. 왜 사용하는것인가?  
https://julien.danjou.info/guide-python-static-class-abstract-methods/ 읽어보자

In [16]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args     # 모든 위치 인수를 다 보여줌

    @staticmethod
    def statmeth(*args):
        return args     # 모든 위치 인수를 다 보여줌

In [17]:
Demo.klassmeth()        # 클래스 메서드는 클래스를 인수로 받음

(__main__.Demo,)

In [18]:
Demo.klassmeth('spam')  

(__main__.Demo, 'spam')

In [19]:
Demo.statmeth       # 함수처럼 동작함

<function __main__.Demo.statmeth(*args)>

In [20]:
Demo.statmeth('spam')

('spam',)

## 9.5 포맷된 출력
- `format()` 내장 함수는 객체를 문자열로 변환한다.
- `format()` 내장 함수는 `__format__()` 특별 메서드를 호출한다.

In [21]:
brl = 1/2.43    # 브라질 레알을 미국 달러로 바꾸는 환율
brl

0.4115226337448559

In [22]:
format(brl, '0.4f')     # '0.4f' 가 포맷 명시자; 소수점 아래 4자리까지 표시

'0.4115'

In [23]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)
#{rate:0.2f}에서 rate는 키워드 인수의 이름, 0.2f는 포맷 명시자
# 포맷명시자에 사용된 표기법을 '포맷 명시 간이 언어'라고 한다.

'1 BRL = 0.41 USD'

In [24]:
print(format(42, 'b'))    # 2진수로 변환
print(format(2/3, '.1%'))    # 백분율로 변환

101010
66.7%


In [25]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')    # 시간을 24시간 형식으로 변환

'20:21:19'

In [26]:
"It's now {:%I:%M %p}".format(now)    # 시간을 12시간 형식으로 변환

"It's now 08:21 PM"

In [27]:
v1 = Vector(3, 4)
format(v1)      # format을 정의하지 않으면, str을 호출한다.

'str-> (3.0, 4.0)'

In [28]:
format(v1, '.3f')

TypeError: unsupported format string passed to Vector.__format__

In [29]:
# 명시자를 사용해 foramt을 출력하고 싶다면 아래와 같이 __format__() 메서드를 구현하면 된다.
from array import array
import math

class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):       # __iter__ 를 통해서 iterable 하게 만들어 준다. -> 언패킹 가능
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return f"repr-> {class_name}({self.x!r}, {self.y!r})"
        # return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return "str-> " + str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)       # hypot = sqrt(x**2 + y**2)) 빗변의 길이를 구함 (즉 벡터간의 거리를 의미함)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __format__(self, fmt_spec: str):
        componets = (format(c, fmt_spec) for c in self)
        return '({}, {})'.format(*componets)

In [30]:
vf = Vector(3, 4)
format(vf, '.2f')

'(3.00, 4.00)'

In [31]:
# 각을 구하기 위한 angle() 메서드를 구현할 수 있다.
def angle(self):
    return math.atan2(self.y, self.x)

In [49]:
# __format__()메서드가 극좌표를 생성하도록 수정해보자

class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):       # __iter__ 를 통해서 iterable 하게 만들어 준다. -> 언패킹 가능
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return f"repr-> {class_name}({self.x!r}, {self.y!r})"
        # return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return "str-> " + str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)       # hypot = sqrt(x**2 + y**2)) 빗변의 길이를 구함 (즉 벡터간의 거리를 의미함)

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __format__(self, fmt_spec: str):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        componets = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*componets)

    def angle(self):
        return math.atan2(self.y, self.x)

In [33]:
print(format(Vector(1, 1), 'p'))
print(format(Vector(1, 1), '.3ep'))
print(format(Vector(1, 1), '0.5fp'))

<1.4142135623730951, 0.7853981633974483>
<1.414e+00, 7.854e-01>
<1.41421, 0.78540>


## 9.6 해시 가능한 Vector2d

In [34]:
v1 = Vector(3, 4)
hash(v1)        # hash 안 됨

TypeError: unhashable type: 'Vector'

In [35]:
set([v1])       # non hashable이기 때문에 set에 넣을 수 없음

TypeError: unhashable type: 'Vector'

In [36]:
# 현재는 이 코드가 실행된다.
print(v1.x, v1.y)

v1.x = 5
print(v1.x, v1.y)       # 벡터를 불변형으로 만들어보자

3.0 4.0
5 4.0


In [37]:
class Vector:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property       # property로 접근해서 읽을 수 있게 만들어준다.
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):      
        return (i for i in (self.x, self.y))

    # 나머지는 생략한대요

In [38]:
vp = Vector(3, 4)
print(vp.x)     # 출력가능
vp.x = 5        # AttributeError: can't set attribute

3.0


AttributeError: can't set attribute

In [45]:
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property       # property로 접근해서 읽을 수 있게 만들어준다.
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):      
        return (i for i in (self.x, self.y))

    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)
    # 나머지는 생략한대요

In [46]:
vh = Vector2d(3, 4)
vh2 = Vector2d(3.1, 4.2)
print(hash(vh), hash(vh2))
print(set([vh, vh2]))       # repr로 출력됨

7 384307168202284039
{<__main__.Vector2d object at 0x10ce30d00>, <__main__.Vector2d object at 0x10ce309a0>}


## 9.7 파이썬에서 비공개 속성과 보호된 속성
`__something`이나 `__something_` 처럼 두 개의 언더바로 시작하고 언더바 없이 또는 하나의 언더바로 끝나도록 정의하면, 파이썬은 언더바와 클래스명을 변수명 앞에 붙여 객체의 `__dict__`에 저장한다. 따라서 Dog 클래스의 경우 __mood는 _Dog__mood가 되고 Beagle 클래스의 경우 _Beagle__mood가 된다. 이러한 파이썬 언어 기능을 이름장식(name mangling)이라고 한다.

In [41]:
v1 = Vector(3, 4)
v1.__dict__

{'_Vector__x': 3.0, '_Vector__y': 4.0}

In [42]:
v1._Vector__x

3.0

## 9.8 `__slots__` 클래스 속성으로 공간 절약하기
파이썬은 객체속성을 각 객체 안의 `__dict__`이라는 딕셔너리 속성에 저장하는데, 딕셔너리는 빠른 접근속도를 제공하기 위해 내부에 해시 테이블을 유지하므로 메모리 사용량 부담이 상당히 크다.  
이런 경우에 `__slots__` 클래스 속성을 이용해서 메모리 사용량을 엄청나게 줄일 수 있다.

`__slots__`을 정의하려면, 이 이름의 클래스 속성을 생성하고 여기에 객체 속성 식별자들을 담은 문자열의 반복형을 할당한다. 불변형인 튜플을 사용하면 `__slots__` 정의를 변경할 수 없음을 알려주므로, 필자는 `__slots__`를 정의할 때 튜플을 즐겨 사용한다고 한다.

In [43]:
class Vector2d():
    __slots__ = ('__x', '__y')

    typecode = 'd'

    # 나머지 코드는 생략한다고 함.

## 9.9 클래스 속성 오버라이드
Vector에 typecode라는 클래스 속성은 `__bytes__()` 메서드에서 두번 사용되는데, 우리는 self.typecode로 그 값을 읽었다. Vector 객체가 그들 자신의 typecode 속성을 가지고 생성된 것이 아니므로, self.typecode는 기본적으로 Vector의 클래스 속성을 가져온다.  
그렇지만 존재하지 않는 속성에 값을 저장하면, 새로운 객체 곳성을 생성하고 동일한 이름의 클래스 속성은 변경하지 않는다. 그 후부터는 객체가 self.typecode를 읽을 때 객체 자체의 typecode를 읽어오기 때문에, 각 객체가 서로 다른 typecode를 가지도록 만들 수 있당

In [55]:
v1 = Vector(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [56]:
len(dumpd)

17

In [57]:
print(v1.typecode)

v1.typecode = 'f'

d


In [58]:
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [59]:
len(dumpf)

9

In [60]:
v1.typecode, Vector.typecode

('f', 'd')